In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin.pandas as pd
from fastai.tabular import *
from utils import isVaildDate, is_more

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-07_11-01-58_31247/logs.
Waiting for redis server at 127.0.0.1:27495 to respond...
Redis failed to start, retrying now.
Waiting for redis server at 127.0.0.1:26210 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 10.0 GB memory using /tmp.


## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user'),
 PosixPath('../test_A/imps_log')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 广告静态数据读入以备用

In [5]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')

In [6]:
ad_static.shape
ad_static.tail()

(502153, 7)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,2019-03-19 03:56:04,21666,21917,18,198,64
502149,665036,2019-02-21 09:08:35,6184,11633,18,232,64
502150,491914,2019-01-25 05:01:10,1809,5278,5,232,64
502151,3990,2019-03-01 08:17:08,4255,3401,18,218,36
502152,222150,2019-02-28 07:56:53,4250,14527,18,94,36


### 历史曝光日志数据文件

In [7]:
col_names = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid',
             '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']

In [8]:
c_sz = 10240

In [9]:
logs = pd.read_csv(log_data/'totalExposureLog.out', sep='\t', header=None, names=col_names, chunksize=c_sz)

- 看下文件信息

In [10]:
!du -h {log_data}/'totalExposureLog.out'

6.2G	../test_A/imps_log/totalExposureLog.out


In [11]:
!cat {log_data}/'totalExposureLog.out' | wc -l

102386695


6.2G 数据 1亿多行数据

- 删除空值 数据量大 考虑删除而不是插值，因为我们不缺数据

- 生成器 读写同时进行（备份）

In [12]:
def invalid_date(df, field):
    ret, l = [], len(df)
    df[field] = pd.to_datetime(df[field], unit='s')  # 转为日期
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    for i in range(l):
        if not isVaildDate(str(df.loc[i, field])):
            ret.append(i)
    # 删除行
    new_df = df.drop(ret, axis=0)
    return new_df

- 获取有效广告 id

In [13]:
ad_id = ad_static['广告id'].unique()

In [ ]:
ad_id.shape

(502153,)

In [ ]:
def invalid_resquest_ad_id(df, field):
    ret, l = [], len(df)
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    for i in range(l):
        if df.loc[i, field] not in ad_id:
            ret.append(i)
    # 删除行
    new_df = df.drop(ret, axis=0)
    return new_df

- 删除非法多值

In [ ]:
def invalid_more_value(df, field):
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    ret = is_more(df, field)
    # 删除行
    new_df = df.drop(ret, axis=0)
    return new_df

- 首先应该删除 `'../data/ad_logs.csv'` 因为接下来我们是以 append 方式写文件

In [ ]:
fn = '../data/ad_logs.csv'
if os.path.exists(fn):
    os.remove(fn)

In [ ]:
for df in logs:
    # 1. 去空值
    df.dropna(axis=0, how='any', inplace=True)
    # 2. 去重 所有列相同
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # 3. 去掉非法时间行
    df = invalid_date(df, '广告请求时间')
    # 4. 删除非法数值 部分处理
    df = invalid_more_value(df, '用户id')
    df = invalid_more_value(df, '广告请求id')
    df = invalid_more_value(df, '曝光广告id')
    df = invalid_more_value(df, '广告位id')
    # 5. 删除非法广告id
    df = invalid_resquest_ad_id(df, '曝光广告id')
    # 6. 数据类型转换
    # 暂无
    df.to_csv('../data/ad_logs.csv', mode='a', index=None, encoding='utf-8', header=False)

- 重新读入 全部数据

In [ ]:
logs1 = pd.read_csv('../data/ad_logs.csv', sep='\t', header=None, names=col_names)

 - 再次去重

In [ ]:
logs1.drop_duplicates(subset=None, keep='first', inplace=True)

In [ ]:
logs1.to_csv('../data/ad_logs.csv', mode='w', index=None, encoding='utf-8')

- 看下文件信息

In [ ]:
!du -h {fn}

In [ ]:
!cat {fn} | wc -l